In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import imageio
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras import backend as k
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import (Dense, Conv2D, BatchNormalization, Activation, MaxPool2D,
                                        Conv2DTranspose, Concatenate, Input, Flatten,
                                    AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.metrics import Recall, Precision, IoU
from tensorflow.keras.applications import ResNet50
from albumentations import HorizontalFlip, GridDistortion, OpticalDistortion, ChannelShuffle, CoarseDropout, CenterCrop, Crop, Rotate

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/people_seg.zip", 'r')
tqdm(zip_ref.extractall("/content/people_segmentation"))
zip_ref.close()

0it [00:00, ?it/s]


In [ ]:
H = 512
W = 512

In [ ]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images", "*.jpg")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))

    #for image, mask in zip(images, masks):
        #print(image, mask)
            #image_1 = cv2.imread(image)

        #cv2.imwrite(r"E:\python\segmentation\Computer Vision\UNET\data\people_segmentation\my_image.png", image_1)
        #mask_1 = cv2.imread(mask)
        #cv2.imwrite(r"E:\python\segmentation\Computer Vision\UNET\data\people_segmentation\mask_1.png", mask_1 * 255)

    split_size = int(len(images) * split)

    X_train, X_test = train_test_split(images, test_size=split_size, random_state=42)
    y_train, y_test = train_test_split(masks, test_size=split_size, random_state=42)

    return (X_train, y_train), (X_test, y_test)

In [ ]:
def augment_data(images, masks, save_aug_path, augment=True):
    H = 512
    W = 512

    for x, y in tqdm(zip(images, masks), total=len(images)):
        #print(x, y)

        name = x.split("/")[-1].split(".")[0]
        #print(name)

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            x2 = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
            y2 = y

            aug = ChannelShuffle(p=1)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            aug = CoarseDropout(p=1, min_holes=3, max_holes=10, max_height=32, max_width=32)
            augmented = aug(image=x, mask=y)
            x4 = augmented["image"]
            y4 = augmented["mask"]

            aug = Rotate(limit=5, p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented["image"]
            y5 = augmented["mask"]

            X = [x, x1, x2, x3, x4, x5]
            Y = [y, y1, y2, y3, y4, y5]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):

            try:
                aug = CenterCrop(H, W, p=1.0)
                augmented = aug(image=i, mask=m)
                i = augmented["image"]
                m = augmented["mask"]

            except Exception as e:
                i = cv2.resize(i, (W, H))
                m = cv2.resize(m, (W, H))

            tem_image_name = f"{name}_{index}.png"
            tem_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_aug_path, "images", tem_image_name)
            mask_path = os.path.join(save_aug_path, "masks", tem_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

In [ ]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intesection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intesection
        x = (intesection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
def load_data_for_train(path):
    """X == images , Y == masks"""
    x = sorted(glob(os.path.join(path, "images", "*.png")))
    y = sorted(glob(os.path.join(path, "masks", "*.png")))

    return x, y

In [ ]:
def read_images(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    #x = cv2.resize(x , (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

In [ ]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    #x = cv2.resize(x , (W, H))
    #x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_images(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [ ]:
def tf_dataset(X, y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [ ]:
def squeeze_and_excite(inputs, ratio=8):
    init = inputs
    filters = init.shape[-1]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation="relu", kernel_initializer="he_normal", use_bias=False)(se)
    se = Dense(filters, activation="sigmoid", kernel_initializer="he_normal", use_bias=False)(se)
    x = init * se
    return x

In [ ]:
def ASPP(inputs):
    #print(inputs.shape)
    shape = inputs.shape
    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(inputs)
    y1 = Conv2D(256, 1, padding="same", use_bias=False)(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    y2 = Conv2D(256, 1, padding="same", use_bias=False)(inputs)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=6)(inputs)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=12)(inputs)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=18)(inputs)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(256, 1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

In [ ]:
def deeplabv3_plus(shape):
    inputs = Input(shape)
    encoder = ResNet50(weights="imagenet", include_top=False, input_tensor=inputs)

    image_features = encoder.get_layer("conv4_block6_out").output
    x_a = ASPP(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    x_b = encoder.get_layer("conv2_block2_out").output
    x_b = Conv2D(filters=48, kernel_size=1, padding="same", use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation("relu")(x_b)

    x = Concatenate()([x_a, x_b])
    x = squeeze_and_excite(x)

    x = Conv2D(filters=256, kernel_size=3, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters=256, kernel_size=3, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = squeeze_and_excite(x)

    x = UpSampling2D((4, 4), interpolation="bilinear")(x)
    x = Conv2D(1, 1)(x)
    x = Activation("sigmoid")(x)

    #print(x.shape)

    model =Model(inputs, x)

    return model

In [ ]:
# if __name__ == "__main__":
#     model = deeplabv3_plus((512, 512, 3))
#     model.summary()

In [ ]:
if __name__ == "__main__":
    np.random.seed(42)

    data_path = "/content/people_segmentation/people_segmentation/"

    (X_train, y_train), (X_test, y_test) = load_data(data_path)
    print(f"train: \t {len(X_train)} - {len(y_train)}")
    print(f"test: \t {len(X_test)} - {len(y_test)}")

    create_dir(data_path + "/new_data/train/images/")
    create_dir(data_path + "/new_data/train/masks/")
    create_dir(data_path + "/new_data/test/images/")
    create_dir(data_path + "/new_data/test/masks/")

    augment_data(X_train, y_train, data_path + "/new_data/train/", augment=True)
    augment_data(X_test, y_test, data_path + "/new_data/test/", augment=False)

train: 	 5111 - 5111
test: 	 567 - 567


100%|██████████| 567/567 [00:16<00:00, 34.27it/s]


In [ ]:
dataset_path = "/content/people_segmentation/people_segmentation/new_data"
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir("/content/drive/MyDrive/" + "/files")

    batch_size = 8
    learning_rate = 1e-4
    num_epochs = 5
    model_path = os.path.join("/content/drive/MyDrive/", "files", "human_image_segmentation_18_epochs_deeplabv3.h5") # Trained for 16 epochs
    csv_path = os.path.join("/content/drive/MyDrive/", "files", "human_image_segmentation_18_epochs_deeplabv3.csv")

    #dataset_path = "new_data"

    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "test")

    X_train, y_train = load_data_for_train(train_path)
    # print("Len training", len(X_train), len(y_train))
    # print(X_train[-1], y_train[-1])

    # extra_element = [item for item in X_train if item not in y_train]

    # if extra_element:
    #     X_train.remove(extra_element[0])
    #     print(f"Extra element {extra_element[0]} found and deleted.")
    # else:
    #     print("No extra element found.")

    print(len(X_train), len(y_train))  # Updated list1 without the extra element

    X_train, y_val = shuffling(X_train, y_train)
    # extra_element = [item for item in y_val if item not in X_train]

    # if extra_element:
    #     y_val.remove(extra_element[0])
    #     print(f"Extra element {extra_element[0]} found and deleted.")
    # else:
    #     print("No extra element found.")

    # print(X_train)
    X_val, y_val = load_data_for_train(valid_path)

    print(f"Training dataset : {len(X_train)} - {len(y_train)}")
    print(f"Validation dataset : {len(X_val)} - {len(y_val)}")

    train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
    valid_dataset = tf_dataset(X_val, y_val, batch=batch_size)

    #for x, y in train_dataset:
        #print(x.shape, y.shape)
        #break

    #model = deeplabv3_plus((H, W, 3))
    model = load_model("/content/drive/MyDrive/files/human_image_segmentation_16_epochs_deeplabv3.h5", compile=False)
    model.compile(
        loss=dice_loss, optimizer=Adam(learning_rate=learning_rate),
                 metrics=[dice_coef, iou, Recall(), Precision()]
    )

    #model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        #TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
    ]

30666 30666
Training dataset : 30666 - 30666
Validation dataset : 567 - 567


In [ ]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

Epoch 1/5
3834/3834 [==============================] - ETA: 0s - loss: 0.4326 - dice_coef: 0.5673 - iou: 0.4244 - recall: 0.8387 - precision: 0.4714
Epoch 1: val_loss improved from inf to 0.43436, saving model to /content/drive/MyDrive/files/human_image_segmentation_16_epochs_deeplabv3.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3834/3834 [==============================] - 3306s 845ms/step - loss: 0.4326 - dice_coef: 0.5673 - iou: 0.4244 - recall: 0.8387 - precision: 0.4714 - val_loss: 0.4344 - val_dice_coef: 0.5657 - val_iou: 0.4010 - val_recall: 0.8801 - val_precision: 0.4262 - lr: 1.0000e-04
Epoch 2/5
3834/3834 [==============================] - ETA: 0s - loss: 0.4284 - dice_coef: 0.5716 - iou: 0.4282 - recall: 0.8377 - precision: 0.4769
Epoch 2: val_loss did not improve from 0.43436
3834/3834 [==============================] - 3207s 836ms/step - loss: 0.4284 - dice_coef: 0.5716 - iou: 0.4282 - recall: 0.8377 - precision: 0.4769 - val_loss: 0.4472 - val_dice_coef: 0.5528 - val_iou: 0.3884 - val_recall: 0.8641 - val_precision: 0.4163 - lr: 1.0000e-04
Epoch 3/5
  70/3834 [..............................] - ETA: 52:27 - loss: 0.2950 - dice_coef: 0.7050 - iou: 0.5664 - recall: 0.8761 - precision: 0.6216

In [ ]:
def save_results(image, mask, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 128
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    mask = mask * 255

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    #y_pred = y_pred * 255

    mask_image = image * y_pred
    y_pred = y_pred * 255

    concatinate_images = np.concatenate([image, line, mask, line, y_pred, line, mask_image], axis=1)

    cv2.imwrite(save_image_path, concatinate_images)

In [ ]:
dataset_path = "/content/drive/MyDrive/people_segmentation/people_segmentation/new_data"
human_image_seg_model_500_epochs = '/content/drive/MyDrive/human_image_segmentation_10_epochs_deeplabv3.h5'
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(dataset_path + "/results")

    with CustomObjectScope({"iou":iou, "dice_coef":dice_coef, "dice_loss":dice_loss}):
        human_image_seg_model = load_model(human_image_seg_model_500_epochs, compile=False)
        #retina_blood_model.summary()

    #dataset_path = "new_data"

    valid_path = os.path.join(dataset_path, "test")

    X_test, y_test = load_data_for_train(valid_path)
    print(f"Test: {len(X_test)} - {len(y_test)}")
    #print(X_test, y_test)

    score = []
    for x, y in tqdm(zip(X_test, y_test), total=len(X_test)):
        #print(x, y)
        name = x.split("/")[-1].split(".")[0]
        print(name)

        image = cv2.imread(x, cv2.IMREAD_COLOR)
        x = image / 255.0
        #print(x.shape)

        x = np.expand_dims(x, axis=0)
        #print(x.shape)

        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        y_pred = human_image_seg_model.predict(x)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred > 0.5
        #print(y_pred.shape)
        y_pred = y_pred.astype(np.int32)
        #y_pred = np.squeeze(y_pred, axis=-1)
        #print(y_pred.shape)

        save_image_path = f"/content/drive/MyDrive/people_segmentation/people_segmentation/new_data/results/{name}.png"

        save_results(image, mask, y_pred, save_image_path)

        mask = mask.flatten()
        y_pred = y_pred.flatten()

        accuracy_value = accuracy_score(mask, y_pred)
        f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary")

        score.append([name, accuracy_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in score]
    score = np.mean(score, axis=0)

    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")


    #df = pd.DataFrame(score, columns=["Image", "Accuracy", "F1_score", "Jaccard", "Recall", "Precision"])
    #df.to_csv("files\\score.csv")

In [ ]:
"""predict"""
human_image_seg_model = "human_image_seg_model_500_epochs"
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir("test_image/mask")

    with CustomObjectScope({"iou":iou, "dice_coef":dice_coef, "dice_loss":dice_loss}):
        human_image_seg_model = load_model(retina_blood_model_path)

    x_data = glob("test_image/iamge/*")

    for path in tqdm(x_data, total=len(x_data)):
        name = path.split("/")[-1].split(".")[0]

        image = cv2.imread(path, cv2.IMREAD_COLOR)
        h, w, _ = image.shape
        x = cv2.resize(image, (W, H))
        x = x / 255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)

        y = human_image_seg_model.predice(x)[0]
        y = cv2.resize(y, (w, h))
        y = np.expand_dims(y, axis=-1)

        masked_image = image * y
        line = np.ones((h, 10, 3)) * 128
        cat_image = np.concatenate([image, line, masked_image], axis=1)
        cv2.imwrite(f"test_image/mask/{name}.png", cat_image)